In [46]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
import torch.nn as nn

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
    #for filename in filenames:
        #print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [47]:
import sys
sys.path.append('/kaggle/usr/lib/metric')

from metric import score

In [48]:
train_sequences = pd.read_csv('/kaggle/input/stanford-rna-3d-folding/train_sequences.csv')

train_labels = pd.read_csv('/kaggle/input/stanford-rna-3d-folding/train_labels.csv')

validation_sequences = pd.read_csv('/kaggle/input/stanford-rna-3d-folding/validation_sequences.csv')

validation_labels = pd.read_csv('/kaggle/input/stanford-rna-3d-folding/validation_labels.csv')

In [49]:
test_labels = pd.read_csv('/kaggle/input/stanford-rna-3d-folding/sample_submission.csv')
test_labels["chain_name"] = test_labels["ID"].str.rsplit("_", n=1).str[0]

# 2. Group by chain and get sequence + length
chain_info = (
    test_labels
    .groupby("chain_name")["resname"]
    .agg(sequence=lambda x: ''.join(x), length='count')
    .reset_index()
)

In [50]:
chain_info

,chain_name,sequence,length
0,R1107,GGGGGCCACAGCAGAAGCGUUCACGUCGCAGCCCCUGUCAGCCAUU...,69
1,R1108,GGGGGCCACAGCAGAAGCGUUCACGUCGCGGCCCCUGUCAGCCAUU...,69
2,R1116,CGCCCGGAUAGCUCAGUCGGUAGAGCAGCGGCUAAAACAGCUCUGG...,157
3,R1117v2,UUGGGUUCCCUCACCCCAAUCAUAAAAAGG,30
4,R1126,GGAAUCUCGCCCGAUGUUCGCAUCGGGAUUUGCAGGUCCAUGGAUU...,363
5,R1128,GGAAUAUCGUCAUGGUGAUUCGUCACCAUGAGGCUAGAUCUCAUAU...,238
6,R1136,GGAUACGUCUACGCUCAGUGACGGACUCUCUUCGGAGAGUCUGACA...,374
7,R1138,GGGAGAGUACUAUUCAGAUGCAGACCGCAAGUUCAGAGCGGUUUGC...,720
8,R1149,GGACACGAGUAACUCGUCUAUCUUCUGCAGGCUGCUUACGGUUUCG...,124
9,R1156,GGAGCAUCGUGUCUCAAGUGCUUCACGGUCACAAUAUACCGUUUCG...,135


In [51]:
train_sequences['length'] = train_sequences['sequence'].str.len()
validation_sequences['length'] = validation_sequences['sequence'].str.len()

In [52]:
unique_values = train_labels['resname'].value_counts()
print("Unique values:", unique_values)

Unique values: resname
G    41450
C    33937
A    32524
U    29178
-        4
X        2
Name: count, dtype: int64


In [53]:
train_labels = train_labels.dropna()

In [54]:
# Extract protein names from 'ID' column
train_labels["RNA_name"] = train_labels["ID"].str.rsplit("_", n=1).str[0]

# Count the number of amino acids per protein
residue_counts = train_labels["RNA_name"].value_counts()

# Filter proteins with fewer than 1000 amino acids
valid_proteins = residue_counts[residue_counts < 1000].index

# Create a new dataframe with only valid proteins
filtered_train_labels = train_labels[train_labels["RNA_name"].isin(valid_proteins)].drop(columns=["RNA_name"])

# Reset index
filtered_train_labels = filtered_train_labels.reset_index(drop=True)

# Display result
print(filtered_train_labels)

              ID resname  resid         x_1         y_1         z_1
0       1SCL_A_1       G      1   13.760000  -25.974001    0.102000
1       1SCL_A_2       G      2    9.310000  -29.638000    2.669000
2       1SCL_A_3       G      3    5.529000  -27.813000    5.878000
3       1SCL_A_4       U      4    2.678000  -24.900999    9.793000
4       1SCL_A_5       G      5    1.827000  -20.136000   11.793000
...          ...     ...    ...         ...         ...         ...
48579  8Z1F_T_62       U     62  112.516998  117.880997  119.245003
48580  8Z1F_T_63       A     63  115.292999  116.571999  114.827003
48581  8Z1F_T_64       C     64  115.857002  114.595001  109.509003
48582  8Z1F_T_65       C     65  113.816002  113.236000  104.339996
48583  8Z1F_T_66       A     66  118.279999  113.883003   98.071999

[48584 rows x 6 columns]


In [55]:
class AttentionWithBias(nn.Module):
    def __init__(self, dim, num_heads):
        super().__init__()
        self.attn = nn.MultiheadAttention(dim, num_heads, batch_first=True)

    def forward(self, x, attn_bias=None):
        attn_output, _ = self.attn(x, x, x, attn_mask=attn_bias)
        return attn_output

"""
class Conv2DNet(nn.Module):
    def __init__(self, input_dim, channels=1):
        super(Conv2DNet, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, channels, kernel_size=1)  # Output shape: (batch, 1, seq_len, seq_len)
        )

    def forward(self, x):
        # x: (batch, seq_len, embed_dim)
        seq_len = x.size(1)
        # Outer product: (batch, seq_len, seq_len, embed_dim)
        outer = torch.einsum("bik,bjk->bij", x, x).unsqueeze(1)
        # outer: (batch, 1, seq_len, seq_len)
        return self.conv(outer).squeeze(1)  # Return (batch, seq_len, seq_len)
"""
class Conv2DNet(nn.Module):
    def __init__(self, seq_len=1024, in_channels=1):
        super(Conv2DNet, self).__init__()
        self.seq_len = seq_len
        self.net = nn.Sequential(
            nn.Conv2d(in_channels, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(16, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 1, kernel_size=1)  # Output shape: (B, 1, L, L)
        )

    def forward(self, pairwise_input):
        # pairwise_input: shape (B, 1, L, L)
        return self.net(pairwise_input).squeeze(1)  # (B, L, L)

In [56]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Squeezeformer Block
class SqueezeformerBlock(nn.Module):
    def __init__(self, dim, num_heads, dropout=0.1):
        super(SqueezeformerBlock, self).__init__()
        self.norm1 = nn.LayerNorm(dim)
        self.attn = nn.MultiheadAttention(embed_dim=dim, num_heads=num_heads, batch_first=True)
        self.norm2 = nn.LayerNorm(dim)
        self.ffn = nn.Sequential(
            nn.Linear(dim, dim * 4),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(dim * 4, dim)
        )

    def forward(self, x, attn_bias=None):
        x_norm = self.norm1(x)
        #attn_output, _ = self.attn(x_norm, x_norm, x_norm, attn_mask=attn_bias)
        attn_output, _ = self.attn(
            x_norm, x_norm, x_norm,
            attn_mask=attn_bias.repeat_interleave(self.attn.num_heads, dim=0) if attn_bias is not None else None
        )

        x = x + attn_output
        x = x + self.ffn(self.norm2(x))
        return x

In [57]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math

class SqueezeformerGRUModel(nn.Module):
    def __init__(self, vocab_size, d_model=128, n_heads=4, num_layers=12, dropout=0.1, max_len=1024, fixed_dist=5.65):
        super(SqueezeformerGRUModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, d_model, padding_idx=vocab_size - 1)
        self.pos_embedding = nn.Parameter(torch.randn(1, max_len, d_model))
        self.fixed_dist = fixed_dist

        self.squeezeformer_blocks = nn.ModuleList([
            SqueezeformerBlock(d_model, n_heads, dropout) for _ in range(num_layers)
        ])

        self.conv2d_net = Conv2DNet(seq_len=max_len, in_channels=1)

        self.gru = nn.GRU(input_size=d_model, hidden_size=d_model, num_layers=1,
                          batch_first=True, bidirectional=True)

        self.direction_head = nn.Linear(d_model * 2, 3)
        self._initialize_direction_head()

    def _initialize_direction_head(self):
        # Initialize weights using kaiming uniform distribution
        nn.init.kaiming_uniform_(self.direction_head.weight, a=math.sqrt(5))
        # Initialize bias with small values
        nn.init.normal_(self.direction_head.bias, std=0.01)

    def forward(self, input_ids):
        B, L = input_ids.shape
        x = self.embedding(input_ids) + self.pos_embedding[:, :L, :]

        x_norm = F.normalize(x, dim=-1)
        pairwise_input = torch.matmul(x_norm, x_norm.transpose(1, 2)).unsqueeze(1)
        attn_bias = self.conv2d_net(pairwise_input)  # (B, L, L)

        for block in self.squeezeformer_blocks:
            x = block(x, attn_bias=attn_bias)

        gru_out, _ = self.gru(x)

        # Compute direction vectors
        direction_vectors = self.direction_head(gru_out[:, :-1])  # (B, L-1, 3)
        
        # Normalize to ensure they lie on the surface of a sphere
        unit_vectors = F.normalize(direction_vectors, dim=-1)

        # Reconstruct coordinates with fixed distance
        coords = [torch.zeros((B, 1, 3), device=unit_vectors.device)]  # start from origin
        for i in range(unit_vectors.shape[1]):
            next_pos = coords[-1] + unit_vectors[:, i:i+1] * self.fixed_dist
            coords.append(next_pos)

        coords = torch.cat(coords, dim=1)  # (B, L, 3)
        return coords

In [58]:
class RNATokenizer:
    def __init__(self):
        self.vocab = {'A': 0, 'C': 1, 'G': 2, 'U': 3}
        self.pad_token = 4
        self.vocab_size = len(self.vocab) + 1  # +1 for PAD token

    def tokenize(self, sequence):
        return [self.vocab.get(res, self.pad_token) for res in sequence]

import torch
from torch.utils.data import Dataset
import numpy as np

class RNADataset(Dataset):
    def __init__(self, df, tokenizer, max_len=1024):
        self.tokenizer = tokenizer
        self.samples = []
        grouped = df.groupby(df['ID'].str.split('_').str[0])  # Group by RNA chain name

        for name, group in grouped:
            sequence = group['resname'].tolist()
            coords = group[['x_1', 'y_1', 'z_1']].values.astype(np.float32)
            coords -= coords[0]  # Shift the first residue to origin


            if len(sequence) > max_len:
                continue  # optionally skip long sequences

            tokenized = tokenizer.tokenize(sequence)
            # Padding
            pad_len = max_len - len(tokenized)
            tokenized += [tokenizer.pad_token] * pad_len
            coords = np.pad(coords, ((0, pad_len), (0, 0)), constant_values=0)

            self.samples.append((tokenized, coords))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        tokens, coords = self.samples[idx]
        return {
            'input_ids': torch.tensor(tokens, dtype=torch.long),
            'target': torch.tensor(coords, dtype=torch.float32),
        }

In [59]:
import torch
import torch.nn as nn

class TMLoss(nn.Module):
    def __init__(self, eps=1e-8):
        super().__init__()
        self.eps = eps

    def forward(self, pred, target):
        """
        pred: (B, L, 3) - predicted 3D coordinates
        target: (B, L, 3) - true 3D coordinates
        """
        B, L, _ = pred.shape
        device = pred.device

        # Distance between each aligned residue pair
        dists = torch.norm(pred - target, dim=-1)  # (B, L)

        # Compute d0 based on the rules
        d0 = torch.zeros(B, device=device)
        for i in range(B):
            L_ref = L  # assuming full alignment for now
            if L_ref < 12:
                d0[i] = 0.3
            elif L_ref < 16:
                d0[i] = 0.4
            elif L_ref < 20:
                d0[i] = 0.5
            elif L_ref < 24:
                d0[i] = 0.6
            elif L_ref < 30:
                d0[i] = 0.7
            else:
                d0[i] = 0.6 * ((L_ref - 0.5) ** 0.5) - 2.5

        # Broadcast d0 for each residue
        d0 = d0.view(B, 1)  # (B, 1)

        tm_per_residue = 1.0 / (1.0 + (dists / (d0 + self.eps)) ** 2)  # (B, L)
        tm_score = tm_per_residue.mean(dim=1)  # (B,) mean over residues

        # We want to maximize TM-score -> so minimize (1 - TM-score)
        loss = 1.0 - tm_score  # (B,)
        return loss.mean()

In [63]:
import torch
import torch.nn.functional as F

model = SqueezeformerGRUModel(vocab_size=5).cuda()
input_ids = torch.randint(0, 4, (1, 128)).cuda()

with torch.no_grad():
    coords, = model(input_ids)

displacements = coords[:, 1:] - coords[:, :-1]  # Shape: (B, L-1, 3)
unit_lengths = displacements.norm(dim=-1)  # Length of each step

# Diagnostic Prints
print(f"Displacements shape: {displacements.shape}")

# Ensure we are not dividing by zero
eps = 1e-8
norms_1 = displacements[:, :-1].norm(dim=-1, keepdim=True) + eps
norms_2 = displacements[:, 1:].norm(dim=-1, keepdim=True) + eps

# Compute cosine of angles between successive steps
angles = (displacements[:, :-1] * displacements[:, 1:]).sum(dim=-1) / (norms_1 * norms_2)

print(angles)

mean_angle = angles.mean().item()
print(f"Mean cosine of angles between successive steps: {mean_angle:.4f}")

Displacements shape: torch.Size([128, 2])
tensor([[ 0.0000e+00, -1.4244e+16, -2.3293e+17,  ...,  3.6363e+20,
          3.8015e+20,  3.9225e+20],
        [ 0.0000e+00, -1.0000e+00, -1.6353e+01,  ...,  2.5529e+04,
          2.6689e+04,  2.7539e+04],
        [ 0.0000e+00, -6.1150e-02, -1.0000e+00,  ...,  1.5611e+03,
          1.6320e+03,  1.6840e+03],
        ...,
        [ 0.0000e+00, -3.9171e-05, -6.4058e-04,  ...,  1.0000e+00,
          1.0454e+00,  1.0787e+00],
        [ 0.0000e+00, -3.7469e-05, -6.1274e-04,  ...,  9.5654e-01,
          1.0000e+00,  1.0318e+00],
        [ 0.0000e+00, -3.6312e-05, -5.9383e-04,  ...,  9.2702e-01,
          9.6913e-01,  1.0000e+00]], device='cuda:0')
Mean cosine of angles between successive steps: 723986213131780096.0000


In [60]:
import torch.nn as nn
from torch.utils.data import DataLoader

tokenizer = RNATokenizer()  # ✅ Define this before using it

# Assuming you already defined SqueezeformerGRUModel
model = SqueezeformerGRUModel(tokenizer.vocab_size)  # Define this next
model = model.cuda()  # Move to GPU if available

dataset = RNADataset(filtered_train_labels, tokenizer, max_len=1024)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

criterion =nn.MSELoss() #TMLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    total_loss = 0.0

    for batch in dataloader:
        input_ids = batch['input_ids'].cuda()
        target = batch['target'].cuda()

        output = model(input_ids)
        loss = criterion(output, target)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    torch.save(model.state_dict(), 'best_model.pt')
    print(f"Epoch {epoch+1}: Loss = {total_loss / len(dataloader):.4f}")

Epoch 1: Loss = 292005.8095
Epoch 2: Loss = 6003.9676
Epoch 3: Loss = 3188.3976
Epoch 4: Loss = 3489.3704
Epoch 5: Loss = 3736.7241
Epoch 6: Loss = 2371.8934
Epoch 7: Loss = 2363.9023
Epoch 8: Loss = 2475.6477
Epoch 9: Loss = 2532.4565
Epoch 10: Loss = 1907.4439


In [ ]:
import torch
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# 1. Grab and tokenize the 1SCL RNA
df_1SCL = train_labels[train_labels["ID"].str.startswith("1SCL_A_")].copy()
seq_1SCL = ''.join(df_1SCL['resname'])  # Original sequence
input_ids = tokenizer.tokenize(seq_1SCL)  # Token IDs

input_tensor = torch.tensor(input_ids).unsqueeze(0).cuda()  # (1, L)

# 2. Run through model
model.eval()
with torch.no_grad():
    pred_coords = model(input_tensor)  # (1, L, 3)
    print("Predicted coords sum:", pred_coords.abs().sum().item())
    print("Output shape:", pred_coords.shape)
pred_coords = pred_coords.squeeze(0).cpu().numpy()  # (L, 3)

# 3. True coordinates
true_coords = df_1SCL[['x_1', 'y_1', 'z_1']].values  # (L, 3)

# 4. Plotting
fig = plt.figure(figsize=(10, 5))
ax1 = fig.add_subplot(121, projection='3d')
ax2 = fig.add_subplot(122, projection='3d')

ax1.plot(true_coords[:, 0], true_coords[:, 1], true_coords[:, 2], label='True', color='green')
ax1.set_title("True RNA Structure")

ax2.plot(pred_coords[:, 0], pred_coords[:, 1], pred_coords[:, 2], label='Predicted', color='blue')
ax2.set_title("Predicted RNA Structure")

for ax in [ax1, ax2]:
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')

plt.tight_layout()
plt.show()

In [ ]:
df_1SCL[['x_pred','y_pred', 'z_pred']]= pred_coords
df_1SCL.head()

In [ ]:
df_1SCL = df_1SCL.copy()
df_1SCL['delta_x'] = df_1SCL['x_1'].diff()
df_1SCL['delta_y'] = df_1SCL['y_1'].diff()
df_1SCL['delta_z'] = df_1SCL['z_1'].diff()
df_1SCL.fillna(0.)
df_1SCL['distance'] = np.sqrt(df_1SCL['delta_x']**2 + df_1SCL['delta_y']**2 + df_1SCL['delta_z']**2)

df_1SCL['delta_x_pred'] = df_1SCL['x_pred'].diff()
df_1SCL['delta_y_pred'] = df_1SCL['y_pred'].diff()
df_1SCL['delta_z_pred'] = df_1SCL['z_pred'].diff()
df_1SCL.fillna(0.)
df_1SCL['distance_pred'] = np.sqrt(df_1SCL['delta_x_pred']**2 + df_1SCL['delta_y_pred']**2 + df_1SCL['delta_z_pred']**2)

In [ ]:
df_1SCL['distance'].hist()

In [ ]:
df_1SCL['distance_pred'].hist()

In [ ]:
# Load the trained model weights before prediction
checkpoint_path = 'best_model.pt'  # adjust this!
model.load_state_dict(torch.load(checkpoint_path))
model.eval()

# Prediction on the validation set

In [ ]:
import torch
from tqdm import tqdm

model.eval()
tokenizer = RNATokenizer()

# Make a copy to modify
validation_predictions = validation_labels[['ID', 'resname', 'resid']]

# Get all unique chain IDs (e.g., "1SCL_A")
chain_ids = validation_predictions['ID'].apply(lambda x: '_'.join(x.split('_')[:1])).unique()

for chain_id in tqdm(chain_ids):
    
    # 1. Extract subsequence for that chain
    chain_df = validation_predictions[validation_predictions['ID'].str.startswith(chain_id)].copy()
    sequence = ''.join(chain_df['resname'].tolist())
    L = len(sequence)

    # 2. Tokenize
    input_ids = tokenizer.tokenize(sequence)
    input_tensor = torch.tensor(input_ids, dtype=torch.long).unsqueeze(0).cuda()  # (1, L)

    # 3. Run through model
    model.eval()
    with torch.no_grad():
        pred_coords = model(input_tensor)  # (1, L, 3)

    # 4. Save back to DataFrame
    pred_coords = pred_coords.squeeze(0).cpu().numpy()[:L]  # (L, 3) - clip if extra padding
    validation_predictions.loc[validation_predictions['ID'].str.startswith(chain_id), ['x_1', 'y_1', 'z_1']] = pred_coords
    validation_predictions.loc[validation_predictions['ID'].str.startswith(chain_id), ['x_2', 'y_2', 'z_2']] = pred_coords
    validation_predictions.loc[validation_predictions['ID'].str.startswith(chain_id), ['x_3', 'y_3', 'z_3']] = pred_coords
    validation_predictions.loc[validation_predictions['ID'].str.startswith(chain_id), ['x_4', 'y_4', 'z_4']] = pred_coords
    validation_predictions.loc[validation_predictions['ID'].str.startswith(chain_id), ['x_5', 'y_5', 'z_5']] = pred_coords

In [ ]:
validation_labels.head()

In [ ]:
validation_predictions.head()

In [ ]:
tm_score = score(validation_labels, validation_predictions.copy(), row_id_column_name='ID')
print(f"Average TM-score: {tm_score:.4f}")

In [ ]:
subm = pd.read_csv('/kaggle/input/stanford-rna-3d-folding/sample_submission.csv')
subm

In [ ]:
import torch
from tqdm import tqdm

model.eval()
tokenizer = RNATokenizer()

# Make a copy to modify
subm_predictions = subm.copy()

# Get all unique chain IDs (e.g., "1SCL_A")
chain_ids = subm_predictions['ID'].apply(lambda x: '_'.join(x.split('_')[:1])).unique()

for chain_id in tqdm(chain_ids):
    
    # 1. Extract subsequence for that chain
    chain_df = subm_predictions[subm_predictions['ID'].str.startswith(chain_id)].copy()
    sequence = ''.join(chain_df['resname'].tolist())
    L = len(sequence)

    # 2. Tokenize
    input_ids = tokenizer.tokenize(sequence)
    input_tensor = torch.tensor(input_ids, dtype=torch.long).unsqueeze(0).cuda()  # (1, L)

    # 3. Run through model
    model.eval()
    with torch.no_grad():
        pred_coords = model(input_tensor)  # (1, L, 3)
        print("Chain ID:", chain_id)
        #print("Sequence:", sequence)
        print("Input tensor:", input_tensor)
        print("Predicted coords sum:", pred_coords.abs().sum().item())
        print("Output shape:", pred_coords.shape)
        print("First 5 direction vectors:\n", pred_coords[0, :5])

    # 4. Save back to DataFrame
    pred_coords = pred_coords.squeeze(0).cpu().numpy()[:L]  # (L, 3) - clip if extra padding
    subm_predictions.loc[subm_predictions['ID'].str.startswith(chain_id), ['x_1', 'y_1', 'z_1']] = pred_coords

In [ ]:
subm_predictions.to_csv('submission.csv', index=False)

In [ ]:
subm_predictions.head()